In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.




In [ ]:
import pandas as pd
import matplotlib as mpl
df = pd.read_csv('/kaggle/input/nba-free-throws/free_throws.csv')

# pm_counter = 0
# pm = {}
# pm_list = []
new_df = df[df['play'].str.contains('1 of 1|1 of 2|1 of 3|2 of 2|2 of 3|3 of 3')]

# for s in new_df.itertuples():
#     unique_id = str(getattr(s, 'game_id'))+getattr(s, 'player')
#     pm_list.append(pm.get(unique_id, 0))
#     pm[unique_id] = pm.get(unique_id, 0) + 1

# new_df.insert(len(new_df.columns), "previous_shots", pm_list)

def convsecs(s):
    s = str(s)
    if len(s) == 4:
        mins = int(s[0:1])
        secs = int(s[2:])
        time = mins * 60 + secs
    else:
        mins = int(s[0:2])
        secs = int(s[3:])
        time = mins * 60 + secs
    return time

class Point:
    def __init__(self,x,y):
        self.x = convsecs(x)
        self.y = y
    
def estimate(player_name, shottype, test):      
    made = []
    missed = []
    made_counter = 0
    for row in new_df.itertuples():
        if shottype in getattr(row, 'play') and getattr(row, 'player')==player_name:
            if getattr(row, 'shot_made')==1:
                made.append(row)
                made_counter += 1
            else:
                missed.append(row)
        
    ratio = made_counter / (len(made)+len(missed))

    #x_made = [getattr(s, 'previous_shots') for s in made]
    y_made = [abs(int(getattr(rows, 'score')[:getattr(rows, 'score').find('-')]) - int(getattr(rows, 'score')[getattr(rows, 'score').find('-')+1:])) for rows in made]
    x_made = [getattr(x, 'time') for x in made]
    made_points = [Point(x,y) for x,y in zip(x_made,y_made)]
    #x_missed = [getattr(s, 'previous_shots') for s in missed]
    x_missed = [getattr(x, 'time') for x in missed]
    y_missed = [abs(int(getattr(rows, 'score')[:getattr(rows, 'score').find('-')]) - int(getattr(rows, 'score')[getattr(rows, 'score').find('-')+1:])) for rows in missed]
    missed_points = [Point(x,y) for x,y in zip(x_missed,y_missed)]

    """
    fig = mpl.pyplot.figure()
    ax = fig.add_subplot(1,1,1)
    ax.scatter([p.x for p in made_points], [p.y for p in made_points], c="green", alpha="0.5", label="made", edgecolors="none", s=20)
    ax.scatter([p.x for p in missed_points], [p.y for p in missed_points], c="red", alpha="0.5", label="missed", edgecolors = "none", s=20)
    mpl.pyplot.xlabel("Number of Free Throws Previously Shot")
    mpl.pyplot.ylabel("Point Differential")
    mpl.pyplot.title(player_name + "Free Throws")
    mpl.pyplot.legend(loc=2)
    mpl.pyplot.show()"""
    def distance(p1,p2,scale):
        return pow(((p1.x-p2.x)*scale)**2+(p1.y-p2.y)**2,0.5)
    
    def nearest_neighbours(p,shots,num,scale):
        return sorted(shots,key=lambda q: distance(p,q,scale))[:num]

    def percentage(neighbours):
        counter = 0
        for n in neighbours:
            if n in made_points:
                counter += 1
        if len(neighbours) == 0:
            return 0
        return counter/len(neighbours)
    
    x_max = max([p.x for p in made_points+missed_points])
    y_max = max([p.y for p in made_points+missed_points])
    try:
        scale = y_max/x_max
    except ZeroDivisionError:
        return 0
    sample_size = int(0.20*(len(made)+len(missed)))
    neighbours = nearest_neighbours(test,made_points+missed_points,sample_size,scale)
    return percentage(neighbours)

testing_set = new_df.sample(int(10000))[0:10000]
predicted_points = 0
predicted_average = 0
actual_points = 0
ds_size = 0
for row in testing_set.itertuples():
    if " of " in getattr(row, 'play'):
        shot_type = getattr(row, 'play')[getattr(row, 'play').find(" of ")-1:getattr(row, 'play').find(" of ")+5]
        y = abs(int(getattr(row, 'score')[:getattr(row, 'score').find('-')]) - int(getattr(row, 'score')[getattr(row, 'score').find('-')+1:]))
        x = getattr(row, 'time')
        #x = getattr(row, 'previous_shots')
        prediction = estimate(getattr(row, 'player'),shot_type,Point(x,y))
        if getattr(row, 'shot_made')==1:
            predicted_points += prediction
            actual_points += 1
        else:
            predicted_points -= prediction
            actual_points -= 1
        ds_size += 1
        predicted_average += prediction
        
        if actual_points != 0:
            print(predicted_points/(predicted_average/ds_size)/actual_points)
